## This is the notebook for the new coffee.csv file we found.

Cleaning and exploration code below...

In [1]:
#!pip install geopy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import re

In [3]:
# Read the csv into a Pandas DataFrame
coffee_df = pd.read_csv('../Resources/coffee.csv')
coffee_df.head()

,all_text,name,rating,roaster,slug,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,desc_4
0,\n\n\n\n \n93\nFlight Coffee Co.\nEthiopia Der...,Ethiopia Deri Kochoha,93,Flight Coffee Co.,/review/ethiopia-deri-kochoha-2,1,0,0,0,0,...,9.0,8.0,9.0,9.0,8.0,NaN,"Bright, crisp, sweetly tart. Citrus medley, ca...",From the Deri Kochoha mill in the Hagere Marya...,A poised and melodic wet-processed Ethiopia co...,NaN
1,\n\n\n\n\n91\nDoi Chaang Coffee\nEspresso\nLoc...,Espresso,91,Doi Chaang Coffee,/review/espresso-14,0,0,0,0,1,...,8.0,NaN,8.0,8.0,8.0,9.0,"Evaluated as espresso. Deeply rich, sweetly ro...",Doi Chaang is a single-estate coffee produced ...,"A rich, resonant espresso from Thailand, espec...",NaN
2,\n\n\n\n \n95\nTemple Coffee and Tea\nKenya Ru...,Kenya Ruthaka Peaberry,95,Temple Coffee and Tea,/review/kenya-ruthaka-peaberry,1,0,0,0,0,...,9.0,8.0,9.0,10.0,8.0,NaN,"Deeply sweet, richly savory. Dark chocolate, p...",Despite challenges ranging from contested gove...,"A high-toned, nuanced Kenya cup, classic in it...",NaN
3,\n\n\n\n \n93\nTemple Coffee and Tea\nEthiopia...,Ethiopia Gora Kone Sidamo,93,Temple Coffee and Tea,/review/ethiopia-gora-kone-sidamo,1,0,0,0,0,...,9.0,8.0,9.0,9.0,8.0,NaN,"Fruit-forward, richly chocolaty. Raspberry cou...",Southern Ethiopia coffees like this one are la...,"A playful, unrestrained fruit bomb of a coffee...",NaN
4,\n\n\n\n\n93\nChoosy Gourmet\nSpecialty Coffee...,Specialty Coffee Blend Espresso,93,Choosy Gourmet,/review/specialty-coffee-blend-espresso,0,0,0,0,0,...,9.0,NaN,8.0,9.0,8.0,9.0,"Evaluated as espresso. Rich, chocolaty, sweetl...",A blend of coffees from Ethiopia (natural-proc...,An espresso blend in which spice notes — in pa...,NaN


In [4]:
# Get a brief summary of DataFrame
coffee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5124 entries, 0 to 5123
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   all_text                5124 non-null   object 
 1   name                    5124 non-null   object 
 2   rating                  5124 non-null   object 
 3   roaster                 5124 non-null   object 
 4   slug                    5124 non-null   object 
 5   region_africa_arabia    5124 non-null   int64  
 6   region_caribbean        5124 non-null   int64  
 7   region_central_america  5124 non-null   int64  
 8   region_hawaii           5124 non-null   int64  
 9   region_asia_pacific     5124 non-null   int64  
 10  region_south_america    5124 non-null   int64  
 11  type_espresso           5124 non-null   int64  
 12  type_organic            5124 non-null   int64  
 13  type_fair_trade         5124 non-null   int64  
 14  type_decaffeinated      5124 non-null   

In [5]:
# drop unnecessary columns 
coffee_df = coffee_df.drop(columns=['all_text', 'roaster', 'slug', 'est_price', 'location'])

In [6]:
coffee_df.head()

,name,rating,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_espresso,type_organic,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,desc_4
0,Ethiopia Deri Kochoha,93,1,0,0,0,0,0,0,0,...,9.0,8.0,9.0,9.0,8.0,NaN,"Bright, crisp, sweetly tart. Citrus medley, ca...",From the Deri Kochoha mill in the Hagere Marya...,A poised and melodic wet-processed Ethiopia co...,NaN
1,Espresso,91,0,0,0,0,1,0,1,0,...,8.0,NaN,8.0,8.0,8.0,9.0,"Evaluated as espresso. Deeply rich, sweetly ro...",Doi Chaang is a single-estate coffee produced ...,"A rich, resonant espresso from Thailand, espec...",NaN
2,Kenya Ruthaka Peaberry,95,1,0,0,0,0,0,0,0,...,9.0,8.0,9.0,10.0,8.0,NaN,"Deeply sweet, richly savory. Dark chocolate, p...",Despite challenges ranging from contested gove...,"A high-toned, nuanced Kenya cup, classic in it...",NaN
3,Ethiopia Gora Kone Sidamo,93,1,0,0,0,0,0,0,0,...,9.0,8.0,9.0,9.0,8.0,NaN,"Fruit-forward, richly chocolaty. Raspberry cou...",Southern Ethiopia coffees like this one are la...,"A playful, unrestrained fruit bomb of a coffee...",NaN
4,Specialty Coffee Blend Espresso,93,0,0,0,0,0,0,1,0,...,9.0,NaN,8.0,9.0,8.0,9.0,"Evaluated as espresso. Rich, chocolaty, sweetl...",A blend of coffees from Ethiopia (natural-proc...,An espresso blend in which spice notes — in pa...,NaN


In [7]:
# Function to split the last word and add it to a new column
def split_last_word(text):
    if isinstance(text, str):
        words = text.split()
        if words:
            return words[-1]
    return None

# Apply the function to create a new column
coffee_df['country_of_origin'] = coffee_df['origin'].apply(split_last_word)

# Display the DataFrame
coffee_df['country_of_origin']

0       Ethiopia
1       Thailand
2          Kenya
3       Ethiopia
4          Kenya
          ...   
5119        None
5120        None
5121        None
5122        None
5123        None
Name: country_of_origin, Length: 5124, dtype: object

In [8]:
# Function to delete everything after the last comma
def delete_after_last_comma(text):
    if isinstance(text, str):
        parts = text.rsplit(',', 1)  # Split at the last comma
        if len(parts) > 1:
            return parts[0].strip()  # Keep only the part before the last comma
    return text

# Apply the function to update the column
coffee_df['origin'] = coffee_df['origin'].apply(delete_after_last_comma)

# Display the updated DataFrame
coffee_df['origin']

0             West Guji Zone, Oromia Region
1                         Northern Thailand
2                      Nyeri growing region
3       Sidamo (also Sidama) growing region
4                        Ethiopia, Colombia
                       ...                 
5119                                    NaN
5120                                    NaN
5121                                    NaN
5122                                    NaN
5123                                    NaN
Name: origin, Length: 5124, dtype: object

In [ ]:
# Create lat and lon columns and use GeoFy library to extract the coordinates of each country

def get_lat_lon_for_country(country_name):
    geolocator = Nominatim(user_agent="country_locator")
    location = geolocator.geocode(country_name)
    
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Create new columns for latitude and longitude
coffee_df['latitude'] = None
coffee_df['longitude'] = None

# Iterate through rows and geocode each country
for index, row in coffee_df.iterrows():
    country_name = row['country_of_origin']  # Replace 'CountryColumnName' with the actual column name in your CSV
    lat_lon = get_lat_lon_for_country(country_name)
    
    if lat_lon:
        coffee_df.at[index, 'latitude'] = lat_lon[0]
        coffee_df.at[index, 'longitude'] = lat_lon[1]

coffee_df.head()